In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #data visualization
import seaborn as sns #data visualization
%matplotlib inline
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Loading the dataset

df = pd.read_csv('enter dataset path')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:

df.isnull().sum()

In [ ]:
plt.figure(figsize=(6, 6))
sns.histplot(df['age'])
plt.title('Distribution of Age')
plt.xlabel('Age')
plt.ylabel('Count')
plt.show()

In [ ]:
categorical_vars = ['gender', 'hypertension', 'heart_disease', 'smoking_history', 'HbA1c_level', 'blood_glucose_level']
# defining the categorical variables to create multiple plots using one code
nrows = 2    #plotting the graphs on the same page
ncols = 3

fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(20, 10))
axs = axs.flatten()

for i, var in enumerate(categorical_vars):                       #creating for loop for all the variables
    ax = axs[i]
    sns.countplot(x=var, data=df, hue='diabetes', ax=ax)
    ax.set_title(f'Distribution of {var}')

plt.tight_layout()   #prevents overlapping
plt.show()

In [ ]:
# converting the categorical values to numeric values
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
from pandas.core.dtypes.common import is_numeric_dtype
for column in df.columns:
    if is_numeric_dtype(df[column]):
        continue
    else:
        df[column]=label.fit_transform(df[column])

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(df.corr(),annot = True )
plt_title = plt.title("Correlation Matrix")


In [ ]:
#handling the imbalanced dataset
from imblearn.combine import SMOTETomek

X = df.drop('diabetes', axis=1)
Y = df['diabetes']

smk = SMOTETomek(random_state=42)
X_resampled, Y_resampled = smk.fit_resample(X, Y)

resampled_df = pd.concat([pd.DataFrame(X_resampled, columns=X.columns),
                          pd.DataFrame(Y_resampled, columns=['diabetes'])], axis=1)
resampled_df.head()

In [ ]:
resampled_df['diabetes'].value_counts()


In [ ]:
# splitting the data into training and testing
from sklearn.model_selection import train_test_split
X = resampled_df.drop('diabetes', axis=1)
Y = resampled_df['diabetes']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)

In [ ]:
import lightgbm as lgb
clf = lgb.LGBMClassifier()
clf.fit(X_train, Y_train)
accuracy = clf.score(X_test, Y_test)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
clf_predict=clf.predict(X_test)
clf_predict
cr=classification_report(Y_test,clf_predict)
print (cr)

In [ ]:
cm = confusion_matrix(Y_test, clf_predict)
sns.heatmap(cm,annot=True)
plt.xlabel("predict value")
plt.ylabel('Actual value')

In [ ]:
# creating a predictive system
input_data = (0,36.0,0,0,1,23.45,5.0,155)
# changing the input_data to numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the array as we are predicting for one instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)


prediction = clf.predict(input_data_reshaped)
print(prediction)

if (prediction[0] == 0):
  print('The person is not diabetic')
else:
  print('The person is diabetic')

In [ ]:
import joblib

model = joblib.load("diabetes_model.pkl")